In [1]:
import os
from urllib.parse import urlparse
import mlflow
import numpy as np
import pandas as pd
import dask.dataframe as dd
from config import settings

In [2]:
from helpers.kaggle_submission import kaggle_submission
from models.pipeline import preprocessing_baseline, show_metrics_baseline, pipeline_after_concat


OSError: Could not find kaggle.json. Make sure it's located in /home/jair/.kaggle. Or use the environment method.

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
from catboost.utils import get_gpu_device_count
from process_and_ml.feature_engineer import fs_for_cat_boost
print('I see %i GPU devices' % get_gpu_device_count())

In [ ]:
import sys
sys.path.append("..")
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs


In [ ]:
from config import settings

In [7]:
where_is = WhereIs(False)

In [8]:
all_paths = where_is.get_paths_list

In [9]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0])


KeyboardInterrupt: 

In [ ]:
customers_df = load_customers(all_paths[1])

In [ ]:
test_file, lnr = load_test_file(all_paths[6])

In [31]:
clean_up = CleanUp(all_paths)

NameError: name 'all_paths' is not defined

In [13]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df, 'test_file': test_file}

In [14]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_dict = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[5], info_df=all_paths[9],attr_df=all_paths[8])

/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].replace(value, np.nan)


In [7]:
path_customer = '../data/cat_dfs/cleaned_customers.csv'
path_azdias = '../data/cat_dfs/cleaned_azdias.csv'

In [19]:
dfs_dict['azdias'].to_csv('../data/cat_dfs/cleaned_azdias.csv', index=False)
dfs_dict['customers_df'].to_csv('../data/cat_dfs/cleaned_customers.csv', index=False)

In [8]:
# run with dask

## Run the same algo with entire dataset
#df_to_model = fs_for_cat_boost(df_population=dfs_dict['azdias'], df_customer=dfs_dict['customers_df'])
df_to_model = dd.concat(
        [dd.read_csv(path_customer).assign(is_customer=1),
         dd.read_csv(path_azdias).assign(is_customer=0)]
    )

In [9]:
len(df_to_model)

1082873

In [6]:
path = '../data/cat_dfs/cleaned_customers_populations.csv'

In [10]:

df_to_model.to_csv(path, single_file=True)


['/home/jair/Documents/arvato_project/data/cat_dfs/cleaned_customers_populations.csv']

In [7]:
dataframe_concatenated = pd.read_csv(path, index_col=[0])


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
dataframe_concatenated.head()

,AGER_TYP,ALTERSKATEGORIE_GROB,ALTER_HH,ANREDE_KZ,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,TITEL_KZ,VERS_TYP,WOHNDAUER_2008,WOHNLAGE,W_KEIT_KIND_HH,ZABEOTYP,is_customer
0,2,4,10.0,1,1.0,0.0,2.0,0.0,1.0,3.0,...,1,6,3,0.0,1,9.0,7.0,6.0,3,1
1,-1,4,11.0,1,NaN,NaN,3.0,0.0,NaN,NaN,...,1,7,3,0.0,1,9.0,NaN,0.0,3,1
2,-1,4,6.0,2,1.0,0.0,1.0,0.0,3.0,7.0,...,1,3,1,0.0,2,9.0,2.0,6.0,3,1
3,1,4,8.0,1,0.0,NaN,0.0,0.0,1.0,7.0,...,4,7,0,0.0,1,9.0,7.0,NaN,1,1
4,-1,3,20.0,1,7.0,0.0,4.0,0.0,3.0,3.0,...,4,5,1,0.0,2,9.0,3.0,2.0,1,1


In [12]:
#dataframe_concatenated = dataframe_concatenated.loc[:, ~dataframe_concatenated.columns.str.match('Unnamed')]

,AGER_TYP,ALTERSKATEGORIE_GROB,ALTER_HH,ANREDE_KZ,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,TITEL_KZ,VERS_TYP,WOHNDAUER_2008,WOHNLAGE,W_KEIT_KIND_HH,ZABEOTYP,is_customer
0,2,4,10.0,1,1.0,0.0,2.0,0.0,1.0,3.0,...,1,6,3,0.0,1,9.0,7.0,6.0,3,1
1,-1,4,11.0,1,NaN,NaN,3.0,0.0,NaN,NaN,...,1,7,3,0.0,1,9.0,NaN,0.0,3,1
2,-1,4,6.0,2,1.0,0.0,1.0,0.0,3.0,7.0,...,1,3,1,0.0,2,9.0,2.0,6.0,3,1
3,1,4,8.0,1,0.0,NaN,0.0,0.0,1.0,7.0,...,4,7,0,0.0,1,9.0,7.0,NaN,1,1
4,-1,3,20.0,1,7.0,0.0,4.0,0.0,3.0,3.0,...,4,5,1,0.0,2,9.0,3.0,2.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082868,-1,3,17.0,2,15.0,0.0,1.0,0.0,NaN,6.0,...,2,2,3,0.0,1,4.0,3.0,3.0,4,0
1082869,-1,2,16.0,1,11.0,0.0,1.0,0.0,4.0,7.0,...,2,4,-1,0.0,-1,9.0,5.0,6.0,6,0
1082870,-1,2,17.0,2,3.0,0.0,0.0,0.0,2.0,5.0,...,7,2,2,0.0,1,5.0,7.0,NaN,4,0
1082871,-1,1,0.0,1,7.0,0.0,1.0,0.0,4.0,2.0,...,5,6,0,0.0,2,9.0,5.0,1.0,5,0


In [ ]:
df_to_model = pd.concat(
    [dfs_dict['customers_df'].assign(is_customer=1),
     dfs_dict['azdias'].assign(is_customer=0)]
)

In [ ]:
## Fazendo o dataframe todo com Dask


In [8]:
cat_features = dataframe_concatenated.select_dtypes(include=['category', 'object']).columns

In [9]:
cat_features

Index(['CAMEO_DEU_2015', 'OST_WEST_KZ'], dtype='object')

In [10]:
features, labels = preprocessing_baseline(dataframe_concatenated,
                                          cat_features=cat_features,
                                          target='is_customer')

In [11]:
X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

In [16]:
params_catboost = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=5,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40)

In [12]:
params_catboost_2 = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=20,
                              one_hot_max_size=4,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC')

### Mlflow data

In [13]:
tags = {'algo':'catboost',
        'datasets':'population_customers',
        'sample':1,
        'cols':cat_features,
        'rows':dataframe_concatenated.shape[0]}
run_name = 'first_cat'


In [4]:
from pathlib import Path
path = (
    Path('.') if Path('.').resolve().name == 'arvato_project'
    else Path('..')
)

In [5]:
path

PosixPath('..')

In [6]:
path_mlflow = path / 'mlruns'
mlflow.set_tracking_uri(str(path_mlflow))
#experiment_id = mlflow.create_experiment('CatBoostClassifier')



In [7]:
mlflow.get_tracking_uri()

'../mlruns'

## Set new experiment

In [8]:
mlflow.set_experiment(experiment_name='CatBoostClassifier')


NameError: name 'cat_features' is not defined

## Defining classificator


In [19]:
catboost = CatBoostClassifier(**params_catboost_2)

##

In [21]:
with mlflow.start_run(tags=tags):
    mlflow.log_params(params_catboost_2)
    catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    metrics_returned = show_metrics_baseline(catboost, features=features, labels=labels)
    mlflow.log_metrics(metrics_returned)

Accuracy Train: 0.7512661425757674
AUC Train: 0.8577617277054371
Accuracy Test: 0.7518577118899717
AUC Test: 0.8584830450433706
Accuracy Valid: 0.7502262499153487
AUC Valid: 0.8583888487640133


In [ ]:
params_catboost = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=5,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40)


## Começa aqui a rodar para a primeira submissao no Kaggle

In [30]:
from models.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool

In [25]:
#Run with just 2 columns
population = pd.read_csv('../data/cleaned_data/azdias_clean.csv', index_col=[0]).sample(frac=0.4, random_state=settings.RANDOM_STATE)
customers = pd.read_csv('../data/cleaned_data/customers_clean.csv', index_col=[0]).sample(frac=0.4, random_state=settings.RANDOM_STATE)

In [79]:
df_to_model = pd.concat(
    [customers.assign(is_customer=1),
     population.assign(is_customer=0)
]
)

In [27]:
cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

In [71]:
cat_features


TypeError: all inputs must be Index

In [80]:
features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')

X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

In [74]:
tags = {'algo':'catboost',
        'datasets':'population_customers',
        'sample':0.2,
        'cols':cat_features,
        'rows':df_to_model.shape[0]}
run_name = 'first_cat'


In [75]:
params_catboost_3 = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=20,
                              one_hot_max_size=4,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC')


In [76]:
catboost = CatBoostClassifier(**params_catboost_3)

In [77]:
with mlflow.start_run(tags=tags):
    mlflow.log_params(params_catboost_3)
    catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    metrics_returned = show_metrics_baseline(catboost, features=features, labels=labels)
    mlflow.log_metrics(metrics_returned)


Accuracy Train: 0.8230135486339231
AUC Train: 0.691796543537512
Accuracy Test: 0.8230160684602598
AUC Test: 0.6841543441025372
Accuracy Valid: 0.8230187924214674
AUC Valid: 0.6912279388484643


In [42]:
test_file, lnr = load_test_file('../data/train_test_files/Udacity_MAILOUT_052018_TEST.csv')

#

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [50]:
test_file_clean = test_file[cat_features]

In [21]:
from process_and_ml.data_wrangler import load_test_file


lnr = test_df['LNR']

In [22]:
test_df.drop(['LNR'], axis=1, inplace=True)

In [51]:
from models.pipeline import cat_features_fill_na

test_df_filled = cat_features_fill_na(test_file_clean, cat_features=['CAMEO_DEU_2015', 'OST_WEST_KZ'])

In [44]:
## kaggle submission

from helpers.kaggle_submission import kaggle_submission

### LOAD MODEL


In [4]:
from models.pipeline import load_catboost_model

catboost = load_catboost_model('catboost_first.cbm')

In [5]:
catboost

In [41]:

kaggle_submission(
    column_lnr=lnr,
    y_pred=catboost.predict_proba(test_file_clean)[:, 1],
    submission_filename='catboost_first_shoot',
    submission_message='CatBoost [30 trees, 3 depth] trained with a sample of a 40% of the data [70/15/15]')



CatBoostError: Invalid cat_features[0] = 9 value: index must be < 2.

#### Save model



In [31]:
test_df['OST_WEST_KZ'].value_counts()

W          26773
O           8433
UNKNOWN     7627
Name: OST_WEST_KZ, dtype: int64

In [28]:
from models.pipeline import save_catboost_model
save_catboost_model(catboost_model=catboost, model_name='catboost_first.cbm', pool_data=Pool(X_train, y_train, cat_features=cat_features))

In [69]:
catboost

In [68]:
y_pred=catboost.predict_proba(test_df_filled)[:, 1]

CatBoostError: Invalid cat_features[0] = 9 value: index must be < 2.

In [83]:
test_to_compare = pd.read_csv('../data/cleaned_data/test_catboost.csv')

In [84]:
set(X_train.columns).difference(test_to_compare.columns)

{'D19_BANKEN_ONLINE_QUOTE_12',
 'D19_GESAMT_ONLINE_QUOTE_12',
 'D19_KONSUMTYP',
 'D19_VERSAND_ONLINE_QUOTE_12',
 'LIFE_AGE',
 'PLZ8_BAUMAX_FAMILY',
 'PLZ8_BAUMAX_bussiness',
 'PRAEGENDE_JUGENDJAHRE_GEN',
 'PRAEGENDE_JUGENDJAHRE_MOV',
 'WEALTH'}

In [86]:
set(test_to_compare.columns).difference(X_train.columns)


{'AGER_TYP',
 'ALTER_HH',
 'CAMEO_DEUINTL_2015',
 'D19_BANKEN_ANZ_12',
 'D19_BANKEN_ANZ_24',
 'D19_BANKEN_DATUM',
 'D19_BANKEN_OFFLINE_DATUM',
 'D19_BANKEN_ONLINE_DATUM',
 'D19_GESAMT_ANZ_12',
 'D19_GESAMT_ANZ_24',
 'D19_GESAMT_DATUM',
 'D19_GESAMT_OFFLINE_DATUM',
 'D19_GESAMT_ONLINE_DATUM',
 'D19_TELKO_ANZ_12',
 'D19_TELKO_ANZ_24',
 'D19_TELKO_OFFLINE_DATUM',
 'D19_TELKO_ONLINE_DATUM',
 'D19_VERSAND_ANZ_12',
 'D19_VERSAND_ANZ_24',
 'D19_VERSAND_DATUM',
 'D19_VERSAND_OFFLINE_DATUM',
 'D19_VERSAND_ONLINE_DATUM',
 'D19_VERSI_ANZ_12',
 'D19_VERSI_ANZ_24',
 'GEBURTSJAHR',
 'KBA05_BAUMAX',
 'KBA13_HERST_SONST',
 'KBA13_KMH_250',
 'KKK',
 'LNR',
 'LP_FAMILIE_GROB',
 'LP_LEBENSPHASE_GROB',
 'LP_STATUS_GROB',
 'PLZ8_BAUMAX',
 'PRAEGENDE_JUGENDJAHRE',
 'REGIOTYP',
 'TITEL_KZ'}